# Set configuration

You need to login to a server and fill the following variables:

In [ ]:
from IPython.core.display import display
import ipywidgets as widgets

session_widget = widgets.Text(placeholder='PHPSESSID', description='Session ID')
server_widget = widgets.Text(value='s1.railnation.de', description='Server URL')
lang_widget = widgets.IntText(value=0, description='Language ID')
invest_widget = widgets.IntSlider(min=0, max=1000000, value=100000, step=10000, description="Maximum Invest")

display(session_widget, server_widget, lang_widget, invest_widget)

In [ ]:
session_id = session_widget.value
server_url = server_widget.value
lang_code = lang_widget.value
maximum_invest = invest_widget.value

%load_ext autoreload
%autoreload 2

In [ ]:
# get user ID
from server import ServerCaller

api = ServerCaller(server_url, session_id)
server = api.call('ServerInfoInterface', 'getInfo', None)
user_id = server['Infos']['activeUser']
print(f'Logged in to {server["Body"]["worldName"]}')

In [ ]:
# get tracks
tracks = api.call('RailInterface', 'getForUser', data=[user_id], short_call=1236)
track_edges = [(track['FromId'], track['ToId']) for track in tracks['Body']]

import networkx as nx

tracks_graph = nx.Graph()
tracks_graph.add_edges_from(track_edges)
print(f'Layed {len(track_edges)} tracks')

In [ ]:
# connected towns
#from city_names import city_names

towns = api.call('LocationInterface', 'getAllTowns', short_call=1236)['Body']

# TODO fix mapping towns -> names
#for i, town in enumerate(towns['Body']):
#    town['Name'] = city_names[str(i+1)][str(lang_code)]

towns = [town for town in towns if town['Id'] in tracks_graph.nodes]
print(f'Connected to {len(towns)} towns')


In [ ]:
# factory properties
factory_properties = api.properties('factories')
factory_growth_properties = api.properties('factory_growth')
factories = api.call('LocationInterface', 'getAllFactories', short_call=1236)['Body']

factories = [factory for factory in factories if factory['Id'] in tracks_graph.nodes]
print(f'Connected to {len(factories)} factories')


In [ ]:
# invest management
import time, random
pp_per_rank = lambda base, rank: max(2, round(base * 0.8 ** (rank - 1)))

for factory in factories:
    print(f"> Opening factory {factory['Id']}, money left: {api.money}")
    if factory['Level'] == 1:
        print('Skipping Level 1 factory') # because we do not know yet how to detect unrevealed factories yet
        continue

    factory_details = api.call('LocationInterface', 'getFactoryDetails', [factory['Id']])['Body']
    invest_rank = api.call('LocationInterface', 'getInvestmentScreenRank', [factory['Id']])['Body']

    # gathering information
    outgoing_good = factory_details['StoragesInfo']['Outgoing'][0]
    current_pp = pp_per_rank(int(factory_properties[outgoing_good]['prestige']),
                             int(invest_rank['UserPositionRank']+1))\
        if invest_rank['UserPositionRank'] >= 0 else 0
    pagination_length = min(5, invest_rank['PlayerCount'])
    invest_screen = api.call('LocationInterface', 'getInvestmentScreen', [factory['Id'], outgoing_good, 0, pagination_length])['Body']

    invest_count = invest_screen['UserInvest']['InvestCount'] if invest_screen['UserInvest'] is not None else 0
    print(f"Currently on rank {invest_rank['UserPositionRank']+1} ({current_pp} PP) with {invest_count} invests")

    # start investing
    if invest_screen['CorpLosesMajority'] != '00000000-0000-0000-0000-000000000000':
        print('Skipping invest because of majority breach')
        continue

    if invest_count < 2:
        print(f"Investing for {invest_screen['Costs']}")
        if invest_screen['Costs'] > api.money or invest_screen['Costs'] > maximum_invest:
            print("Not enough money or above invest limit")
            continue
        invest_count = api.call('LocationInterface', 'invest', [factory['Id']])['Body']
    time.sleep(random.randint(1, 3))
print('-- Done --')


In [ ]:
# tabular factory details
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

data = [{
        'Id': factory['Parameters'][0],
        'Level': factory['Body']['Level'],
        'GrowthProgress': factory['Body']['WorkloadInfo']['GrowthPoints'] / 1000 - (factory['Body']['Level'] - 1),
        'GrowthTrend': factory['Body']['WorkloadInfo']['Trend'],
     } for factory in factory_details]

df = pd.DataFrame(data)
df


In [ ]:
# train spotters
spotters = api.call('TrainSpotterInterface', 'getWaitingAndCollected', short_call=1236)
print(f'Found {len(spotters["Body"]["Waiting"]["TrainSpotter"])} train spotters')

collected_spotters = list(api.bulk_call('TrainSpotterInterface', 'collect', spotters['Body']['Waiting']['TrainSpotter']))
print(f'Collected {len(collected_spotters)} train spotters')
